In [1]:
import os
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/c

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('videogame_vine_reviews').getOrCreate()

# Load Amazon Data into DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_games = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)
video_games.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
filtered_video_games = video_games.select(['star_rating', 'helpful_votes', 'total_votes', "vine", "verified_purchase"])
filtered_video_games.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          1|            0|          1|   N|                Y|
|          3|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          1|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [7]:
# Filter for greater than 15 total votes
total_votes = filtered_video_games.filter(filtered_video_games["total_votes"] >= 15)
total_votes.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           21|         34|   N|                N|
|          1|           21|         35|   N|                Y|
|          5|           16|         17|   N|                Y|
|          1|           13|         15|   N|                N|
|          5|           14|         18|   N|                Y|
|          1|          147|        175|   N|                Y|
|          4|           13|         15|   N|                N|
|          1|           14|         31|   N|                Y|
|          2|           55|         60|   N|                N|
|          1|           51|         65|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [9]:
# Filter for greater than 50% helpful vote percentage
helpful_votes = total_votes.filter(total_votes["helpful_votes"]/total_votes["total_votes"] >= 0.5)
helpful_votes.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           21|         34|   N|                N|
|          1|           21|         35|   N|                Y|
|          5|           16|         17|   N|                Y|
|          1|           13|         15|   N|                N|
|          5|           14|         18|   N|                Y|
|          1|          147|        175|   N|                Y|
|          4|           13|         15|   N|                N|
|          2|           55|         60|   N|                N|
|          1|           51|         65|   N|                Y|
|          1|           14|         16|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



# Stats

In [10]:
# Paid reviews
paid = helpful_votes.filter(helpful_votes['vine']== 'Y')
paid.describe().show(10)

+-------+------------------+------------------+-----------------+----+-----------------+
|summary|       star_rating|     helpful_votes|      total_votes|vine|verified_purchase|
+-------+------------------+------------------+-----------------+----+-----------------+
|  count|               128|               128|              128| 128|              128|
|   mean|         4.1328125|         43.734375|        49.765625|null|             null|
| stddev|1.0146260608259017|58.727783117043664|62.29375221449283|null|             null|
|    min|                 1|                10|              102|   Y|                N|
|    max|                 5|                97|               88|   Y|                N|
+-------+------------------+------------------+-----------------+----+-----------------+



In [12]:
# Unpaid reviews
unpaid = helpful_votes.filter(helpful_votes['vine']== 'N')
unpaid.describe().show()

+-------+------------------+------------------+------------------+-----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+------------------+------------------+------------------+-----+-----------------+
|  count|             59338|             59338|             59338|59338|            59338|
|   mean|  3.32474973878459|36.579190400754996| 43.45653712629344| null|             null|
| stddev|1.6433223951567544| 98.37277416134035|106.78366939897671| null|             null|
|    min|                 1|                10|               100|    N|                N|
|    max|                 5|               999|               999|    N|                Y|
+-------+------------------+------------------+------------------+-----+-----------------+



In [14]:
#Percentage of 5 star reviews among vine reviews
paid_five_star_number = paid[paid['star_rating']== 5].count()
paid_number = paid.count()
percentage_five_star_vine = round(float(paid_five_star_number) / float(paid_number),4)
print(f'Number of paid reviews: {paid_number}')
print(f'Number of paid five star reviews: {paid_five_star_number}')
print(f'Percantage of paid reviews that are five stars: {percentage_five_star_vine * 100}%')

Number of paid reviews: 128
Number of paid five star reviews: 62
Percantage of paid reviews that are five stars: 48.44%


In [15]:
#Percentage of 5 star reviews among non-vine reviews
unpaid_five_star_number = unpaid[unpaid['star_rating']== 5].count()
unpaid_number = unpaid.count()
percentage_five_star_non_vine = round(float(unpaid_five_star_number) / float(unpaid_number),4)
print(f'Number of unpaid reviews: {unpaid_number}')
print(f'Number of unpaid five star reviews: {unpaid_five_star_number}')
print(f'Percantage of paid reviews that are five stars: {percentage_five_star_non_vine * 100}%')

Number of unpaid reviews: 59338
Number of unpaid five star reviews: 22583
Percantage of paid reviews that are five stars: 38.06%
